- 범주형 컬럼에 onehotencoder 적용
- 원 데이터에 변환한 컬럼 붙이기

In [2]:
# 데이터 수집
import pandas as pd
# OnehotEncoding 처리
from sklearn.preprocessing import OneHotEncoder
# 모델 학습
from sklearn.linear_model import LogisticRegression
# 모델 평가
from sklearn.metrics import accuracy_score

df_titanic = pd.read_csv('../../datasets/titanic_disaster_train.csv')
df_titanic.head(3) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df_titanic['Sex'].value_counts() # 남녀 각각 몇명인지 카운트트

Sex
male      577
female    314
Name: count, dtype: int64

In [5]:
# 데이터 전처리(결측치 처리)
df_titanic.dropna(inplace=True) # 결측값 행 제거

In [6]:
df_titanic.info() # 결측치 제거 되었는지 확인

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


In [7]:
df_titanic['Sex'].value_counts() # 결측치 제거 후 남녀 몇 명인지 카운트

Sex
male      95
female    88
Name: count, dtype: int64

In [8]:
# OnehotEncoding 처리

df_titanic['Sex'].unique(), df_titanic['Sex'].shape

(array(['female', 'male'], dtype=object), (183,))

In [9]:
df_titanic[['Sex']].shape

(183, 1)

In [11]:
onehot_sex = OneHotEncoder()
onehot_sex.fit(df_titanic[['Sex']])

OneHotEncoder()

In [12]:
onehot_sex.categories_

[array(['female', 'male'], dtype=object)]

In [14]:
encoded_sex = onehot_sex.transform(df_titanic[['Sex']]).toarray()
# encoded_sex
encoded_sex.shape, type(encoded_sex)

((183, 2), numpy.ndarray)

In [15]:
sex_name_list = onehot_sex.get_feature_names_out(['Sex'])

df_people = pd.DataFrame(data = encoded_sex
                         , columns = sex_name_list
                         , index = df_titanic.index)

df_people.head(2)

,Sex_female,Sex_male
1,1.0,0.0
3,1.0,0.0


In [16]:
df_titanic_concated = pd.concat([df_titanic, df_people], axis = 1)
df_titanic_concated.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0.0


In [17]:
features = df_titanic_concated.drop(columns = ['Survived', 'PassengerId', 'Name'
                                    , 'Sex', 'Ticket', 'Cabin', 'Embarked'
                                    , 'Pclass'])
label = df_titanic_concated['Sex']
features.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')

In [18]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         183 non-null    float64
 1   SibSp       183 non-null    int64  
 2   Parch       183 non-null    int64  
 3   Fare        183 non-null    float64
 4   Sex_female  183 non-null    float64
 5   Sex_male    183 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 10.0 KB


In [19]:
# 모델 학습
logisticRegression = LogisticRegression()
logisticRegression.fit(X = features, y = label)

LogisticRegression()

In [20]:
logisticRegression.coef_, logisticRegression.intercept_

(array([[ 1.19473337e-02, -6.67607287e-02,  7.67031574e-03,
         -1.39732881e-03, -3.26747626e+00,  3.26745524e+00]]),
 array([-0.24244357]))

In [21]:
# 모델평가

predicts = logisticRegression.predict(features)
predicts

array(['female', 'female', 'male', 'female', 'female', 'male', 'male',
       'male', 'female', 'male', 'male', 'female', 'male', 'female',
       'male', 'male', 'male', 'male', 'male', 'male', 'female', 'male',
       'female', 'male', 'male', 'male', 'female', 'male', 'male',
       'female', 'male', 'male', 'female', 'female', 'female', 'male',
       'female', 'female', 'male', 'female', 'male', 'male', 'female',
       'male', 'female', 'male', 'male', 'female', 'female', 'male',
       'female', 'female', 'male', 'female', 'female', 'male', 'female',
       'female', 'female', 'female', 'female', 'female', 'female',
       'female', 'female', 'male', 'male', 'male', 'female', 'male',
       'male', 'female', 'female', 'female', 'female', 'female', 'male',
       'male', 'male', 'female', 'female', 'female', 'male', 'male',
       'male', 'female', 'male', 'male', 'male', 'male', 'male', 'male',
       'male', 'male', 'female', 'male', 'female', 'male', 'male',
       'female', '

In [22]:
accuracy_score(y_true=label, y_pred=predicts)

1.0

In [25]:
predicts_2 = logisticRegression.predict(features[:2])
predicts_2

array(['female', 'female'], dtype=object)

In [26]:
logisticRegression.predict_proba(features[:2])

array([[0.96166446, 0.03833554],
       [0.96204727, 0.03795273]])